# RUBIX Pipeline for NIHAO

The RUBIX pipeline has been extended to support **NIHAO simulations**. This notebook demonstrates how to use the pipeline to transform NIHAO data into mock IFU cubes. Similar to Illustris, the pipeline executes data transformation in a linear process.

## How to Use the Pipeline
1) Define a `config`
2) Set up the `pipeline yaml`
3) Run the RUBIX pipeline
4) Analyze the mock data

## Step 1: Configuration

Below is an example configuration for running the pipeline with NIHAO data. Replace `path` and `halo_path` with the paths to your NIHAO snapshot and halo files.
This configuration supports quick testing by using only a subset of the data (1000 particles).

In [ ]:
#NBVAL_SKIP
config = {
    "pipeline": {"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "NihaoHandler",
        "args": {
            "particle_type": ["stars", "gas"],
            "save_data_path": "data",
        },
        "load_galaxy_args": {"reuse": True},
        "subset": {"use_subset": True, "subset_size": 1000},
    },
    "simulation": {
        "name": "NIHAO",
        "args": {
            "path": "/path/to/nihao/snapshot",
            "halo_path": "/path/to/nihao/halo/file",
        },
    },
    "output_path": "output",

    "telescope": {
        "name": "MUSE",
        "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
        "lsf": {"sigma": 0.5},
        "noise": {"signal_to_noise": 1, "noise_distribution": "normal"},
    },
    "cosmology": {"name": "PLANCK15"},
    "galaxy": {
        "dist_z": 0.1,
        "rotation": {"type": "edge-on"},
    },
    "ssp": {
        "template": {"name": "BruzualCharlot2003"},
    },
}

## Step 2: Pipeline YAML

To run the RUBIX pipeline, you need a YAML file (stored in `rubix/config/pipeline_config.yml`) that defines which functions are used during the execution of the pipeline. This shows the example pipeline YAML to compute a stellar IFU cube.

```yaml
calc_ifu:
  Transformers:
    rotate_galaxy:
      name: rotate_galaxy
      depends_on: null
      args: []
      kwargs:
        type: "edge-on"
    filter_particles:
      name: filter_particles
      depends_on: rotate_galaxy
      args: []
      kwargs: {}
    spaxel_assignment:
      name: spaxel_assignment
      depends_on: filter_particles
      args: []
      kwargs: {}
    reshape_data:
      name: reshape_data
      depends_on: spaxel_assignment
      args: []
      kwargs: {}
    calculate_spectra:
      name: calculate_spectra
      depends_on: reshape_data
      args: []
      kwargs: {}
    scale_spectrum_by_mass:
      name: scale_spectrum_by_mass
      depends_on: calculate_spectra
      args: []
      kwargs: {}
    doppler_shift_and_resampling:
      name: doppler_shift_and_resampling
      depends_on: scale_spectrum_by_mass
      args: []
      kwargs: {}
    calculate_datacube:
      name: calculate_datacube
      depends_on: doppler_shift_and_resampling
      args: []
      kwargs: {}
    convolve_psf:
      name: convolve_psf
      depends_on: calculate_datacube
      args: []
      kwargs: {}
    convolve_lsf:
      name: convolve_lsf
      depends_on: convolve_psf
      args: []
      kwargs: {}
    apply_noise:
      name: apply_noise
      depends_on: convolve_lsf
      args: []
      kwargs: {}
```

## Step 3: Run the Pipeline

Now, simply execute the pipeline with the following code.

In [ ]:
#NBVAL_SKIP
from rubix.core.pipeline import RubixPipeline
pipe = RubixPipeline(config)

rubixdata = pipe.run()

## Step 4: Visualize the Mock Data

### Plot a Spectrum for a Single Spaxel

In [ ]:
#NBVAL_SKIP
import jax.numpy as jnp
import matplotlib.pyplot as plt

wave = pipe.telescope.wave_seq
spectra = rubixdata["datacube"]

plt.plot(wave, spectra[12, 12, :])
plt.title("Spectrum of Spaxel [12, 12]")
plt.xlabel("Wavelength [Å]")
plt.ylabel("Flux")
plt.show()

### Create a Spatial Image from the Data Cube

In [ ]:
#NBVAL_SKIP
visible_indices = jnp.where((wave >= 4000) & (wave <= 8000))

visible_spectra = spectra[:, :, visible_indices[0]]
image = jnp.sum(visible_spectra, axis=2)

plt.imshow(image, origin="lower", cmap="inferno")
plt.colorbar(label="Integrated Flux")
plt.title("Spatial Image from Data Cube")
plt.show()

## DONE!

Congratulations, you have successfully processed NIHAO simulation data using the RUBIX pipeline.